In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.weightstats import ttest_ind

base_path = Path().cwd().parent / "data/stats"

# One- and two-sample statistics
One- and two-sample statistics are statistics where we compare one group to some predifined mean or distribution or compare whether two gropus are different. There are parametric and non-parametric tests.

## The t-test
The t-test is perhaps one of the simplest statistical tests you can run. Usually in electrophysiology we are comparing two different (independent) groups, comparing two different timepoints or drug treatment within a group (paired, not independent) or comparing whether a single group is different from a predefined mean (one-sample t-test) which can be used for a paired t-test. You can run a t-test using Scipy or Statsmodels in Python. I always recommend getting the confidence intervals of the difference between the two groups. Usually when the confidence intervals of the difference do not include 0 a statistically significant effect. The Further from 0 that your interval is the larger the effect size. Since the p-value is dependent on sample size this helps interpret the p-value. If you are running a t-test between two different groups there are several things to consider.
1. Are your data normally distributed? Yes, go to 2.
2. Are your two groups independent? Yes, use z-test, t-test or Welch's t-test. No, use paired t-test.
3. Is the variance of your two groups the same? Yes, use standard t-test. No, use Welch's t-test.
4. Are you sample sizes unequal? Use Welch's t-test
5. How large are your sample sizes? Larger than ~30 use the z-test

### The output
You will get the test statistic, a p-value and the degrees of freedom. You can also get confidence intervals. For simple tests like t-tests I prefer to get bootstrapped confidence intervals.

### Scipy vs Statmodels
I prefer Scipy for the t-tests but, I will include the code for Statsmodels as well. 

### Two-sample t-test with equal variances

#### Scipy

In [ ]:
data = pd.read_csv(base_path / "ttest.csv", header=None)
one = data.loc[data[1] == "one", 0]
two = data.loc[data[1] == "two", 0]
output = stats.ttest_ind(one, two)
print(output)
print(output.confidence_interval())

#### Statsmodels

In [ ]:
output = ttest_ind(one, two, alternative="two-sided", usevar="pooled")
print(output)

### Comparing two-sample t-test with unqual variances with and without Welch's correction
You will see that without Welch's correction that p-value and the degrees of freedom (df) is inflated. One thing to note is that Welch's correct will converge towards a standard t-test if the variances and sample sizes are equal.

#### Scipy

In [ ]:
data = pd.read_csv(base_path / "ttest_welch.csv", header=None)
one = data.loc[data[1] == "one", 0]
two = data.loc[data[1] == "two", 0]
output = stats.ttest_ind(one, two, equal_var=True)
output_w = stats.ttest_ind(one, two, equal_var=False)
print(output, output_w, sep="\n")

#### Statsmodels

In [ ]:
output = ttest_ind(one, two, alternative="two-sided", usevar="unequal")
output_w = ttest_ind(one, two, alternative="two-sided", usevar="equal")
print(output, output_w, sep="\n")

### One tailed t-test
One tailed t-tests are used when you have a specific hypothesis about the direction of your effect. This needs to be choosen before you collect your data. An example of when this would be used is if you have a treatment that "needs" to be more effective than a previous treatment or if you are replicating data and aleady have a hypothesis about the direction of your effect. The direction of you effect is determine by why group you have as one and two. If you expect one to be larger than two (positive differece between the groups) you want larger/greater. If you expect one to be smaller than two (negative difference between the groups) the you want less/smaller.

### Paired t-test
The paired t-test is used when you have a subject that has two measures and you want to compare if the two measures are different. Paired data (aka within-subject data) violates the independence assumption of a traditional t-test. Also paired t-test will increase your statistical power by reducing between subject variance since each subject essentially acts as its own control.`

#### Scipy

In [ ]:
data = pd.read_csv(base_path / "ttest.csv", header=None)
one = data.loc[data[1] == "one", 0]
two = data.loc[data[1] == "two", 0]
output = stats.ttest_rel(one, two)
print(output)

#### Statsmodels
Statsmodels does not have a one sample t-test. Since the one-sample t-test is pretty simple to run we will manually compute it here. If you want to determine whether you difference is different from some other value than zero then you can replace 0 with that value.

In [ ]:
diffs = one - two
m_diff = diffs.mean() - 0
se = diffs.std(ddof=1) / np.sqrt(diffs.size)
t_value = m_diff / se
p = stats.t.sf(np.abs(t_value), df=diffs.size - 1) * 2  # two-sided test
print(f"statistic: {t_value}, pvalue: {p}, df: {diffs.size - 1}")

## Rank-based two-sample statistics
There are a variety of non-parametric two-sample rank-based statistics such as Mann-Whitney and Wilcoxon signed-rank test and the bootstrap test. Some things you should be aware of before using rank-based tests. If your data is continuous, try transforming your data before running a rank-based test. I often see people fall back to Mann-Whitney for non-normal data. However, in most cases a simple log-transform would have been enough to make the data fit the assumptions of the t-test and its variants. Another is that for rank-based tests in biological sciences we generally want to test if the location or scale is different but not both because that makes interpreting our data hard. To do so we are most interested in the median. If your two samples each have a distribution (same gamma) with different shapes **AND** dispersion you may not have a difference in medians (due to location or scale) which is what we really want to test with rank-based tests. The test is still valid you just cannot say where the difference in your samples are. If you do not know the terms location, shape, scale or dispersion check out the [Distributions](=distributions) chapter. Non-parametric rank-based statistics really have almost as many assumptions as parametric-based statistics. Rank-based statistics can also be sensitive the number of ties or the number of same values between your two group. Rank-based statistics are also not a substitute for parametric tests when your sample size is small (n=3 for each group). The last case where rank-based tests may be useful is when you have outliers

### Mann-Whitney U
Mann-Whitney is a rank-based statistical test for two indepedent samples. Rather than compare the mean difference, Mann-Whitney essentially scores how many values are larger than in the other group and creates a U score which is used to generate a p-value. For the Mann-Whitney test to be valid your data must come from the same distribution type otherwise interpreting your what difference your significant p-value is showing is next to impossible. When reporting the Mann-Whitney p-values you should also report the difference between the medians (or the median of each group), the Hodges-Lehmann estimator which is like an effects size measures for the Mann-Whitney test and the probability of superiority.

In [ ]:
from itertools import product

def hodges_lehmann(group1, group2):
    """Median of all pairwise differences"""
    pairwise_diffs = [y - x for x, y in product(group1, group2)]
    return np.median(pairwise_diffs)

def probability_of_superiority(group1, group2):
    """P(Y > X) - common language effect size"""
    pairs = [(x, y) for x, y in product(group1, group2)]
    return np.mean([y > x for x, y in pairs])

data = pd.read_csv(base_path / "ttest.csv", header=None)
one = data.loc[data[1] == "one", 0]
two = data.loc[data[1] == "two", 0]
output = stats.mannwhitneyu(one, two,)


### Brunner-Munzel
Brunner-Munzel is a rank-based tests with very few assumptions. It is robust to outliers, it can be used when the shape or dispersion of a distributions are different. It tests whether the probability of getting large values in both groups is the same